#1. DATA WORK

###### 파이토치에는 데이터 작업을 위한 기본 요소 2가지가 있다.     
###### 1) torch.utils.data.DataLoader 
###### 2) torch.utils.data.Dataset

Dataset 은 Sample 과 Label(정답)을 저장하고
DataLoader 는 Dataset 을 순회 가능한 객체(iterable)로 감싼다.

* iterable object - 데이터를 하나하나씩 처리할 수 있는 객체들을 순회가능 객체라 칭한다. 
##### set, list, dict, tuple, generator 등은 모두 순회가능(iterable) 객체이다.
##### 어떤 클래스를 iterable하게 하려면 __iter__와 __next__ 멤버함수를 구현해야한다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 

##### PyTorch 에는 TorchText, TorchVision 및 TorchAudio 와 같이 도메인 특화 라이브러리를 데이터셋과 함께 제공하고 있다.
##### torchvision.datasets 모듈은 CIFAR, COCO 등과 같은 다양한 실제 비전(vision) 데이터에 대한 Dataset 을 포함하고 있다. 
##### 모든 TorchVision Dataset 은 sample 과 label 을 각각 변경하기 위한 transform 과 target_transform 의 두 인자를 포함한다.

In [2]:
# 공개 데이터셋에서 학습 데이터를 다운로드
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 다운로드
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



##### Dataset 을 DataLoader 의 인자로 전달한다. 이는 데이터셋을 iterable object 로 감싸고, batch, sampling, shuffle and multiprocess data loading 을 지원한다. 
##### 여기서 배치 사이즈는 64로 정의한다. 즉, DataLoader 객체의 각 요소는 64개의 feature 과 label 을 batch 로 반환한다.

In [3]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y : {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


#2. MAKE MODEL

###### PyTorch 에서 신경망(NN) 모델은 nn.Module 을 상속받는 클래스(class)를 생성하여 정의한다. 
###### __init__ 함수에서 신경망의 layer 들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정한다.
###### 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelerate) 한다. 

In [4]:
# 학습에 사용할 CPU 나 GPU 장치를 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
  def __init__(self):
      super(NeuralNetwork, self).__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(28*28, 512),
          nn.ReLU(),
          nn.Linear(512,512),
          nn.ReLU(),
          nn.Linear(512,10)
      )

  def forward(self,x):
     x = self.flatten(x)
     logits = self.linear_relu_stack(x)
     return logits

model = NeuralNetwork().to(device)
print(model) 

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#3. OPTIMIZE MODEL VARIABLES

###### 모델을 학습하려면 손실함수(loss function) 과 옵티마이저(optimizer) 가 필요하다. 

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

##### 각 학습단계(training loop)에서 모델은(batch로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정한다. 

* 역전파: 먼저 계산 결과와 정답의 오차를 구해 이 오차에 관여하는 값들의 가증치를 수정하여 오차가 작아지는 방향으로 일정 횟수를 반복해 수정하는 방법이다. 오차역전파 또는 오류역전파라고도 불린다.

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산 
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")



##### 모델이 학습하고 있는지를 확인하기 위해 테스트 데이터셋으로 모델의 성능을 확인한다.

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#4. CHECK ACCURACY

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
  
print("Done!")

Epoch 1
-----------------------
loss: 2.297476 [    0 / 60000]
loss: 2.294436 [ 6400 / 60000]
loss: 2.273790 [12800 / 60000]
loss: 2.267439 [19200 / 60000]
loss: 2.261389 [25600 / 60000]
loss: 2.228013 [32000 / 60000]
loss: 2.232843 [38400 / 60000]
loss: 2.203398 [44800 / 60000]
loss: 2.196645 [51200 / 60000]
loss: 2.158477 [57600 / 60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.160412 

Epoch 2
-----------------------
loss: 2.169701 [    0 / 60000]
loss: 2.160508 [ 6400 / 60000]
loss: 2.109022 [12800 / 60000]
loss: 2.116909 [19200 / 60000]
loss: 2.061934 [25600 / 60000]
loss: 2.016528 [32000 / 60000]
loss: 2.026607 [38400 / 60000]
loss: 1.961346 [44800 / 60000]
loss: 1.967510 [51200 / 60000]
loss: 1.873756 [57600 / 60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.884910 

Epoch 3
-----------------------
loss: 1.923462 [    0 / 60000]
loss: 1.888695 [ 6400 / 60000]
loss: 1.784651 [12800 / 60000]
loss: 1.810614 [19200 / 60000]
loss: 1.685264 [25600 / 60000]
loss: 1.656214 [32000 / 6

#5. SAVE & OPEN MODEL

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [ 
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
